In [ ]:
!python -m pip install --upgrade pip

In [ ]:
!python -m pip install --upgrade pip
!python -m pip install "torch>=2.1.0" torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118


!python -m pip install "transformers>=4.41.0,<5.0.0" "huggingface-hub>=0.34.0" datasets

!python -m pip install faster_whisper

!python -m pip install optimum==1.26.1

!python -m pip install openai==1.55.0

Looking in indexes: https://download.pytorch.org/whl/cu118
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 30.2 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.0/38.0 MB 54.1 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 80.3 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 MB 43.6 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6/6 [faster_whisper]
  Attempting uninstall: openai
    Found existing installation: openai 1.109.1
    Uninstalling openai-1.109.1:
      Successfully uninstalled openai-1.109.1


In [ ]:
import torch, transformers,librosa, soundfile, sentencepiece, faster_whisper, optimum, openai,importlib.metadata

print("Torch version:", torch.__version__)
print("Transformers version:", transformers.__version__)
print("Librosa version:", librosa.__version__)
print("Optimum version:", importlib.metadata.version('optimum'))
print("Whisper version:",faster_whisper.__version__)
print("OpenAI version:", openai.__version__)

Torch version: 2.8.0+cu126
Transformers version: 4.57.1
Librosa version: 0.11.0
Optimum version: 1.26.1
Whisper version: 1.2.1
OpenAI version: 1.55.0


In [3]:
from huggingface_hub import login

login(token="enter your huggingface token")

In [1]:
from google.colab import files

print("Please select your audio file (wav format recommended).")
uploaded = files.upload()

audio_file = list(uploaded.keys())[0]
print(f"Selected file: {audio_file}")

Please select your audio file (wav format recommended).


Saving test.m4a to test.m4a
Selected file: test.m4a


In [2]:
import librosa
import numpy as np


audio, sr = librosa.load(audio_file, sr=16000, mono=True)
audio = audio.astype(np.float32)
print(f"Audio loaded. Shape: {audio.shape}, Sample Rate: {sr}")

/tmp/ipython-input-3635911904.py:5: UserWarning: PySoundFile failed. Trying audioread instead.
  audio, sr = librosa.load(audio_file, sr=16000, mono=True)
/usr/local/lib/python3.12/dist-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)


Audio loaded. Shape: (155990,), Sample Rate: 16000


In [4]:
from transformers import Wav2Vec2FeatureExtractor, Wav2Vec2ForSequenceClassification


model_name = "hughlan1214/Speech_Emotion_Recognition_wav2vec2-large-xlsr-53_240304_SER_fine-tuned1.1"


feature_extractor = Wav2Vec2FeatureExtractor.from_pretrained(model_name)
model = Wav2Vec2ForSequenceClassification.from_pretrained(model_name)

print("Model and feature extractor loaded successfully.")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/212 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/1.26G [00:00<?, ?B/s]

Model and feature extractor loaded successfully.


In [5]:
import librosa
import torch
import torch.nn.functional as F


audio, sr = librosa.load(audio_file, sr=16000, mono=True)


inputs = feature_extractor(audio, sampling_rate=sr, return_tensors="pt", padding=True)


with torch.no_grad():
    logits = model(**inputs).logits


probs = F.softmax(logits, dim=-1)


labels = ["angry", "disgust", "fear", "happy", "neutral", "sad"]


pred_id = torch.argmax(probs, dim=-1).item()
voice_emotion = labels[pred_id]
confidence = probs[0, pred_id].item()

print(f"Predicted Emotion: {voice_emotion}, Confidence: {confidence:.2f}")

/tmp/ipython-input-2048822773.py:6: UserWarning: PySoundFile failed. Trying audioread instead.
  audio, sr = librosa.load(audio_file, sr=16000, mono=True)
/usr/local/lib/python3.12/dist-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)


Predicted Emotion: neutral, Confidence: 0.60


In [8]:
from faster_whisper import WhisperModel


whisper_model_name = "small"


model = WhisperModel(whisper_model_name, device="cuda" if torch.cuda.is_available() else "cpu")

print(f"Loaded Faster-Whisper model: {whisper_model_name}")

model.bin:   0%|          | 0.00/484M [00:00<?, ?B/s]

vocabulary.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

config.json: 0.00B [00:00, ?B/s]

Loaded Faster-Whisper model: small


In [9]:
segments, info = model.transcribe(audio_file, beam_size=5)

print(f"Detected language: {info.language}")
print("\nTranscription result:\n")


transcript = ""
for segment in segments:
    print(f"[{segment.start:.2f}s → {segment.end:.2f}s]: {segment.text}")
    transcript += segment.text + " "


transcript = transcript.strip()

Detected language: en

Transcription result:

[0.00s → 6.68s]:  I'm actually sad today but I'm feeling a bit optimistic of one and lost a few
[6.68s → 11.36s]:  competitions but I'm keeping it positive


In [10]:

from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

model_name = "j-hartmann/emotion-english-distilroberta-base"


tokenizer = AutoTokenizer.from_pretrained(model_name)
text_emotion_model = AutoModelForSequenceClassification.from_pretrained(model_name)


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
text_emotion_model = text_emotion_model.to(device)

print(f"Loaded text emotion model: {model_name}")

tokenizer_config.json:   0%|          | 0.00/294 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/329M [00:00<?, ?B/s]

Loaded text emotion model: j-hartmann/emotion-english-distilroberta-base


In [12]:
import torch.nn.functional as F
import numpy as np


try:
    transcript
except NameError:
    transcript = input("Enter the text to analyze emotion: ")


inputs = tokenizer(transcript, return_tensors="pt", truncation=True, padding=True).to(device)


with torch.no_grad():
    outputs = text_emotion_model(**inputs)
    probs = F.softmax(outputs.logits, dim=-1).cpu().numpy()[0]


labels = text_emotion_model.config.id2label
pred_id = int(np.argmax(probs))
pred_label = labels[pred_id]
confidence = float(probs[pred_id])

print(f"Predicted Emotion: {pred_label} (Confidence: {confidence:.4f})")

print("\nAll emotion probabilities:")
for i, score in enumerate(probs):
    print(f"  {labels[i]}: {score:.4f}")

Predicted Emotion: joy (Confidence: 0.7566)

All emotion probabilities:
  anger: 0.0014
  disgust: 0.0014
  fear: 0.0013
  joy: 0.7566
  neutral: 0.0054
  sadness: 0.2288
  surprise: 0.0052


In [13]:

def fuse_emotions(voice_emotion, voice_conf, text_emotion, text_conf):
    """
    Simple weighted fusion:
    - Takes voice and text emotions with their confidence scores
    - Returns the final predicted emotion and its confidence
    """

    weights = {"voice": voice_conf, "text": text_conf}


    if voice_emotion == text_emotion:
        fused_emotion = voice_emotion
        fused_conf = min(1.0, (voice_conf + text_conf) / 2 + 0.1)
    else:

        if voice_conf >= text_conf:
            fused_emotion = voice_emotion
            fused_conf = voice_conf * 0.8 + text_conf * 0.2
        else:
            fused_emotion = text_emotion
            fused_conf = text_conf * 0.8 + voice_conf * 0.2

    return fused_emotion, fused_conf


final_emotion, final_confidence = fuse_emotions(voice_emotion, confidence, labels, score)
print(f"Fused Emotion: {final_emotion}, Confidence: {final_confidence:.2f}")


Fused Emotion: neutral, Confidence: 0.61


In [14]:


from transformers import AutoTokenizer, AutoModelForCausalLM
import torch


model_name = "gpt2-medium"

print("Loading model:", model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
model.eval()

print("Model loaded on", device)

if tokenizer.pad_token_id is None:
      tokenizer.pad_token_id=tokenizer.eos_token_id



Loading model: gpt2-medium


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/718 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.52G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

Model loaded on cpu


In [ ]:

llm_prompt = f"""
You are an assistant that interprets spoken user input.
Your task is to produce a JSON with two fields: "literal_speech" and "likely_intent".

Inputs:
- Transcript: "{transcript}"
- Fused emotion (from voice+text): "{final_emotion}" (confidence: {final_confidence:.2f})
- Voice-only emotion: "{voice_emotion}" (confidence: {confidence:.2f})
- Text-only emotion: "{labels}" (confidence: {score})

Instructions:
1. "literal_speech" should be exactly what the user said (short).
2. "likely_intent" should explain in 3-5 sentences what the user actually intended to say,
   taking into account the fused emotion and its confidence.
3. The "likely_intent" must always be longer than the transcript and elaborate on
   the user's possible feelings, intentions, or context.
4. Output ONLY valid JSON. Do not add any text outside the JSON object.
5. Example output:
{{
    "literal_speech": "I am not sad",
    "likely_intent": "The user probably meant that they are feeling a bit upset but still trying to stay positive. They might have faced some disappointments but are maintaining optimism. Overall, they are expressing a mixture of sadness and hope, showing resilience."
}}
"""


inputs = tokenizer(llm_prompt, return_tensors="pt")
if torch.cuda.is_available():
    inputs = {k: v.to("cuda") for k, v in inputs.items()}


with torch.no_grad():
    outputs = model.generate(
        **inputs,
        max_new_tokens=200,
        do_sample=True,
        temperature=0.7,
        pad_token_id=tokenizer.eos_token_id
    )


raw_output = tokenizer.decode(outputs[0], skip_special_tokens=True)

try:

    json_start = raw_output.find("{")
    json_end = raw_output.rfind("}") + 1
    parsed_output = json.loads(raw_output[json_start:json_end])
except:
    parsed_output = {"literal_speech": transcript, "likely_intent": "Unable to infer refined intent."}


print("LLM OUTPUT")
print("Literal speech:", parsed_output.get("literal_speech"))
print("Likely intent :", parsed_output.get("likely_intent"))
